Importing necessary modules

In [6]:
import cv2
import numpy as np
import requests
import shutil

Get model files for pose networks

In [7]:
def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [11]:
proto_file = "pose_deploy.prototxt"
weights_file = "pose_iter_584000.caffemodel"
weights_file_url = "http://posefs1.perception.cs.cmu.edu/Users/tsimon/Projects/coco/data/models/coco/pose_iter_440000.caffemodel"

Load neural network

In [32]:
try:
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
except:
    #download weights file
    file = download_file(weights_file_url)
    net = cv2.dnn.readNetFromCaffe(proto_file, file)

if net.empty():
    raise ValueError("Failed to load the model. Check the file paths.")
    

In [39]:
input_layer_names = net.getLayerNames()
id = net.getLayerId(input_layer_names[0])
input_blob_shape = net.

print("Input blob shape:", input_blob_shape)

error: OpenCV(4.10.0) /io/opencv/modules/dnn/src/net_quantization.cpp:268: error: (-6:Unknown error code -6) Net isn't quantized in function 'getInputDetails'


Video capture or read from file

In [13]:
capture = True

Visualization structure

In [14]:
BODY_PARTS = {0: "Nose", 1: "Neck", 2: "Right Shoulder", 3: "Right Elbow", 
              4: "Right Wrist", 5: "Left Shoulder", 6: "Left Elbow", 
              7: "Left Wrist", 8: "Right Hip", 9: "Right Knee", 
              10: "Right Ankle", 11: "Left Hip", 12: "Left Knee", 
              13: "Left Ankle", 14: "Right Eye", 15: "Left Eye", 
              16: "Right Ear", 17: "Left Ear"}
POSE_PAIRS = [[1, 2], [1, 5], [2, 3], [3, 4], [5, 6], [6, 7], 
              [1, 8], [8, 9], [9, 10], [1, 11], [11, 12], [12, 13], 
              [0, 14], [14, 16], [0, 15], [15, 17]]

In [ ]:
Video capture code

In [31]:
if capture:
    cap = cv2.VideoCapture("test1.mkv")
    threshold = 0.1
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_width = frame.shape[1]
        frame_height = frame.shape[0]

        # Prepare the frame for input
        inp_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (368, 368), (0, 0, 0), swapRB=False, crop=False)
        #print(inp_blob)
        if frame is None or frame.size == 0:
            print("Empty frame detected!")
            continue
        net.setInput(inp_blob)

        # Get the output
        output = net.forward()
                    # Extract points
        points = []
        for i in range(len(BODY_PARTS)):
            heat_map = output[0, i, :, :]
            _, conf, _, point = cv2.minMaxLoc(heat_map)
            x = int((frame_width * point[0]) / output.shape[3])
             y = int((frame_height * point[1]) / output.shape[2])
            if conf > threshold:
                points.append((x, y))
                cv2.circle(frame, (x, y), 5, (0, 255, 0), thickness=-1)
            else:
                points.append(None)
                    # Draw lines for pose
        for pair in POSE_PAIRS:
            part_from = pair[0]
            part_to = pair[1]

            if points[part_from] and points[part_to]:
                cv2.line(frame, points[part_from], points[part_to], (0, 255, 255), 2)
                cv2.circle(frame, points[part_from], 5, (0, 0, 255), thickness=-1)
                cv2.circle(frame, points[part_to], 5, (0, 0, 255), thickness=-1)


        # Show the result
        cv2.imshow("Pose Estimation", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[ERROR:0@12386.735] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12386.735] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12386.735] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'



Error forwarding blob [[[[0.7372549  0.7411765  0.74509805 ... 0.73333335 0.6901961
    0.6901961 ]
   [0.7372549  0.7411765  0.74509805 ... 0.7254902  0.7176471
    0.70980394]
   [0.7372549  0.74509805 0.7529412  ... 0.72156864 0.7411765
    0.72156864]
   ...
   [0.10588235 0.10588235 0.10980392 ... 0.36862746 0.62352943
    0.9764706 ]
   [0.10588235 0.10588235 0.10980392 ... 0.38039216 0.49803922
    0.9647059 ]
   [0.10588235 0.10588235 0.11764706 ... 0.3764706  0.41960785
    0.88235295]]

  [[0.7372549  0.7411765  0.74509805 ... 0.7019608  0.70980394
    0.7058824 ]
   [0.7372549  0.7411765  0.74509805 ... 0.7058824  0.7058824
    0.7019608 ]
   [0.7372549  0.74509805 0.7529412  ... 0.70980394 0.7019608
    0.7058824 ]
   ...
   [0.06666667 0.06666667 0.0627451  ... 0.34509805 0.63529414
    0.99215686]
   [0.06666667 0.06666667 0.0627451  ... 0.33333334 0.47843137
    0.98039216]
   [0.06666667 0.0627451  0.07058824 ... 0.30588236 0.3529412
    0.8666667 ]]

  [[0.7372549  0.7

[ERROR:0@12386.941] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12386.941] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12386.941] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@12386.964] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12386.964] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12386.964] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Asserti

Error forwarding blob [[[[0.7372549  0.7372549  0.7411765  ... 0.70980394 0.7019608
    0.7058824 ]
   [0.7372549  0.7372549  0.7411765  ... 0.70980394 0.7019608
    0.7058824 ]
   [0.7372549  0.7372549  0.7411765  ... 0.7058824  0.7019608
    0.7019608 ]
   ...
   [0.09411765 0.09411765 0.08627451 ... 0.35686275 0.5647059
    0.9647059 ]
   [0.09411765 0.09411765 0.09411765 ... 0.36078432 0.5176471
    0.9647059 ]
   [0.10196079 0.10196079 0.09803922 ... 0.34901962 0.5176471
    0.9843137 ]]

  [[0.7372549  0.7372549  0.7411765  ... 0.70980394 0.7019608
    0.7058824 ]
   [0.7372549  0.7372549  0.7411765  ... 0.70980394 0.7019608
    0.7058824 ]
   [0.7372549  0.7372549  0.7411765  ... 0.7058824  0.7019608
    0.7019608 ]
   ...
   [0.0627451  0.0627451  0.06666667 ... 0.3254902  0.6
    0.9843137 ]
   [0.0627451  0.0627451  0.07450981 ... 0.32941177 0.54509807
    0.9843137 ]
   [0.07058824 0.06666667 0.07843138 ... 0.3254902  0.53333336
    0.98039216]]

  [[0.7372549  0.7372549  0.

[ERROR:0@12387.155] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12387.155] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12387.155] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@12387.223] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12387.223] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12387.223] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Asserti

...

[ERROR:0@12416.234] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12416.234] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12416.234] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@12416.309] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12416.309] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12416.309] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Asserti

Error forwarding blob [[[[0.6862745  0.69411767 0.6901961  ... 0.654902   0.654902
    0.6431373 ]
   [0.6862745  0.69411767 0.6901961  ... 0.6666667  0.654902
    0.65882355]
   [0.68235296 0.6862745  0.6862745  ... 0.65882355 0.65882355
    0.6627451 ]
   ...
   [0.10980392 0.10588235 0.10980392 ... 1.         1.
    1.        ]
   [0.11372549 0.10980392 0.10196079 ... 1.         1.
    1.        ]
   [0.10588235 0.10196079 0.09803922 ... 1.         1.
    1.        ]]

  [[0.67058825 0.6745098  0.6784314  ... 0.6431373  0.654902
    0.6431373 ]
   [0.67058825 0.6745098  0.6784314  ... 0.64705884 0.63529414
    0.6392157 ]
   [0.67058825 0.6784314  0.6862745  ... 0.6392157  0.6392157
    0.6431373 ]
   ...
   [0.05882353 0.05882353 0.05490196 ... 0.9843137  0.9843137
    0.9843137 ]
   [0.06666667 0.0627451  0.05882353 ... 0.9843137  0.9843137
    0.9843137 ]
   [0.05882353 0.05490196 0.05490196 ... 0.9843137  0.9843137
    0.9843137 ]]

  [[0.6666667  0.67058825 0.6745098  ... 0.647

[ERROR:0@12416.462] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12416.463] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12416.463] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@12416.541] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12416.541] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12416.541] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Asserti

Error forwarding blob [[[[0.6862745  0.69411767 0.6901961  ... 0.6666667  0.68235296
    0.6627451 ]
   [0.6862745  0.69411767 0.6901961  ... 0.6784314  0.6745098
    0.654902  ]
   [0.68235296 0.6862745  0.6862745  ... 0.6784314  0.6627451
    0.6509804 ]
   ...
   [0.11372549 0.09803922 0.09411765 ... 1.         1.
    1.        ]
   [0.09803922 0.10196079 0.07058824 ... 1.         1.
    1.        ]
   [0.09411765 0.11372549 0.09019608 ... 1.         1.
    1.        ]]

  [[0.67058825 0.6745098  0.6784314  ... 0.63529414 0.6431373
    0.6431373 ]
   [0.67058825 0.6745098  0.6784314  ... 0.64705884 0.6431373
    0.6431373 ]
   [0.67058825 0.6784314  0.6862745  ... 0.64705884 0.64705884
    0.6509804 ]
   ...
   [0.0627451  0.0627451  0.05882353 ... 0.9843137  0.9843137
    0.9843137 ]
   [0.0627451  0.05882353 0.0627451  ... 0.9843137  0.9843137
    0.9843137 ]
   [0.0627451  0.05882353 0.05882353 ... 0.9843137  0.9843137
    0.9843137 ]]

  [[0.6666667  0.67058825 0.6745098  ... 0.

[ERROR:0@12416.706] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@12416.707] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 368 368 ]
[ERROR:0@12416.707] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

